In [ ]:
# default_exp benchmarking

# benchmarking

> This module contains new evaluation protocol for UBC Phototour local patch dataset

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import numpy as np
import gc
import os
from fastprogress.fastprogress import progress_bar
from scipy.spatial.distance import cdist, hamming
from sklearn.metrics.pairwise import paired_distances
from sklearn.metrics import average_precision_score

def evaluate_mAP_snn_based(descriptors:np.array,
                           labels:np.array,
                           img_labels:np.array, 
                           path_to_save_mAP: str,
                           backend:str ='numpy', distance:str ='euclidean'):
    '''Function to calculate mean average precision, over per-image based matching using Lowe SNN ratio.'''
    if os.path.isfile(path_to_save_mAP):
        print (f"Found saved results {path_to_save_mAP}, loading")
        res = np.load(path_to_save_mAP)
        return res
    backends = ['numpy', 'pytorch-cuda']
    if backend not in backends:
        raise ValueError(f'backend {backend} should one of {backends}')
    possible_distances = ['euclidean', 'hamming']
    if distance == 'euclidean':
        p=2
    elif distance == 'hamming':
        p=0
    else:
        raise ValueError(f'distance {distance} should one of {possible_distances}')    
    APs = []
    unique_img_labels = sorted(np.unique(img_labels))
    for img_idx in progress_bar(unique_img_labels):
        current_batch = img_labels == img_idx
        cur_descs = descriptors[current_batch]
        if backend == 'pytorch-cuda':
            import torch
            dev = torch.device('cpu')
            try:
                if torch.cuda.is_available():
                    dev = torch.device('cuda')
            except:
                dev = torch.device('cpu')
            cur_descs = torch.from_numpy(cur_descs).to(dev).float()  
        cur_labels = labels[current_batch]
        NN = cur_labels.shape[0]
        pos_labels_repeat = np.broadcast_to(cur_labels.reshape(1,-1),(NN,NN))
        pos_mask = (pos_labels_repeat == pos_labels_repeat.T)
        pos_mask_not_anchor = pos_mask != np.eye(NN, dtype=np.bool)
        neg_idx = np.zeros((NN), dtype=np.int32)
        if NN > 1000: # To avoid OOM, we will find hard negative in batches
            bs1 = 128
            nb = (NN // bs1)  
            for i in range(nb):
                st = i*bs1
                fin = min(NN, (i+1)*bs1)
                if fin == st:
                    break
                if backend == 'pytorch-cuda':
                    dm = torch.cdist(cur_descs[st:fin], cur_descs, p=p) +\
                            1000.0 * torch.from_numpy(pos_mask[st:fin]).to(device=dev, dtype=cur_descs.dtype) + \
                            1000.0 * torch.eye(NN, device=dev, dtype=torch.bool)[st:fin].float()
                    min_neg_idxs = torch.min(dm, axis=1)[1].cpu().numpy()
                else:
                    dm = cdist(cur_descs[st:fin], cur_descs, metric=distance) +\
                            1000.0 * pos_mask[st:fin] + \
                            1000.0 * np.eye(NN, dtype=np.bool)[st:fin]
                    min_neg_idxs = np.argmin(dm, axis=1)
                neg_idx[st:fin] = min_neg_idxs
        # We want to create all possible anchor-positive combinations
        pos_idxs = np.broadcast_to(np.arange(NN).reshape(1,-1),(NN,NN))[pos_mask_not_anchor]
        anc_idxs = np.nonzero(pos_mask_not_anchor)[0]
        pos_mask = None
        neg_idxs = neg_idx[anc_idxs]
        if backend == 'pytorch-cuda':
            pos_dists = torch.nn.functional.pairwise_distance(cur_descs[anc_idxs], cur_descs[pos_idxs], p=p).detach().cpu().numpy()
            neg_dists = torch.nn.functional.pairwise_distance(cur_descs[anc_idxs], cur_descs[neg_idxs], p=2).detach().cpu().numpy()
        else:
            if distance == 'hamming':
                pos_dists = paired_distances(cur_descs[anc_idxs], cur_descs[pos_idxs], metric=hamming)
                neg_dists = paired_distances(cur_descs[anc_idxs], cur_descs[neg_idxs], metric=hamming)
            else:
                pos_dists = paired_distances(cur_descs[anc_idxs], cur_descs[pos_idxs], metric=distance)
                neg_dists = paired_distances(cur_descs[anc_idxs], cur_descs[neg_idxs], metric=distance)
        correct = pos_dists <= neg_dists
        snn = np.minimum(pos_dists,neg_dists) / np.maximum(pos_dists,neg_dists)
        snn[np.isnan(snn)] = 1.0
        ap = average_precision_score(correct, 1-snn)
        APs.append(ap)
        pos_mask = None
        pos_mask_not_anchor = None
        cur_descs = None
        pos_labels_repeat = None
        dm = None
        gc.collect()
    res = np.array(APs).mean()
    if not os.path.isdir(os.path.dirname(path_to_save_mAP)):
        os.makedirs(os.path.dirname(path_to_save_mAP))
    np.save(path_to_save_mAP, res)
    return res

In [ ]:
#export
from brown_phototour_revisited.extraction import *
from collections import defaultdict

def load_cached_results(desc_name: str,
                    learned_on: list = ['3rdparty'],
                    path_to_save_dataset:str = './dataset/',
                    path_to_save_descriptors: str = './descriptors/',
                    path_to_save_mAP: str = './mAP/',
                    patch_size: int = 32):
    '''Function, which checks, if the descriptor was already evaluated, and if yes - loads it'''
    subsets = ['liberty', 'notredame', 'yosemite']
    results = defaultdict(dict)
    for train_ds in learned_on:    
        for subset in subsets:
            if train_ds == '3rdparty':
                load_path = f'{path_to_save_mAP}/{desc_name}_PS{patch_size}_3rdparty_{subset}.npy'
            else:
                load_path = f'{path_to_save_mAP}/{desc_name}_PS{patch_size}_learned{train_ds}_{subset}.npy'
            if os.path.isfile(load_path):
                print (f"Found saved results {load_path}, loading")
                mAP = np.load(load_path)
                results[train_ds][subset] = mAP
                print (f'{desc_name} trained on {learned_on} PS = {patch_size} mAP on {subset} = {mAP:.5f}')
    return results

In [ ]:
#export
from brown_phototour_revisited.extraction import *
from collections import defaultdict

def full_evaluation(models,
                    desc_name: str,
                    path_to_save_dataset:str = './dataset/',
                    path_to_save_descriptors: str = './descriptors/',
                    path_to_save_mAP: str = './mAP/',
                    patch_size: int = 32, 
                    device: str = 'cpu',
                    backend='numpy',
                    distance='euclidean'):
    '''Function, which performs descriptor extraction and evaluation on all datasets.
    models can be either torch.nn.Module or dict with keys ['liberty', 'notredame', 'yosemite'],
    denoting datasets, each model was trained on resp.'''
    subsets = ['liberty', 'notredame', 'yosemite']
    if type(models) is dict:
        results = load_cached_results(desc_name,
                                      [x for x in models.keys()],
                                      path_to_save_dataset,
                                      path_to_save_descriptors,
                                      path_to_save_mAP,
                                      patch_size)
        for learned_on, model in models.items():
            for subset in subsets:
                if subset == learned_on:
                    continue
                if learned_on in results:
                    if subset in results:
                        continue
                try:
                    desc_dict = extract_pytorchinput_descriptors(model,
                                    desc_name + '_' + learned_on,
                                    subset = subset, 
                                    path_to_save_dataset = path_to_save_dataset,
                                    path_to_save_descriptors = path_to_save_descriptors,
                                    patch_size = patch_size, 
                                    device = device)
                except:
                    desc_dict = extract_numpyinput_descriptors(model,
                                    desc_name + '_' + learned_on,
                                    subset= subset, 
                                    path_to_save_dataset = path_to_save_dataset,
                                    path_to_save_descriptors = path_to_save_descriptors,
                                    patch_size = patch_size)                    
                mAP = evaluate_mAP_snn_based(desc_dict['descriptors'],
                             desc_dict['labels'], 
                             desc_dict['img_idxs'],
                             path_to_save_mAP=f'{path_to_save_mAP}/{desc_name}_PS{patch_size}_learned{learned_on}_{subset}.npy',
                             backend=backend,
                             distance=distance)
                results[learned_on][subset] = mAP
                print (f'{desc_name} trained on {learned_on} PS = {patch_size} mAP on {subset} = {mAP:.5f}')
    else:
        model = models
        results = load_cached_results(desc_name,
                                      ['3rdparty'],
                                      path_to_save_dataset,
                                      path_to_save_descriptors,
                                      path_to_save_mAP,
                                      patch_size)
        for subset in subsets:
            if '3rdparty' in results:
                if subset in results['3rdparty']:
                    continue
            try:
                desc_dict = extract_pytorchinput_descriptors(model,
                                desc_name + '_3rdparty' ,
                                subset= subset, 
                                path_to_save_dataset = path_to_save_dataset,
                                path_to_save_descriptors = path_to_save_descriptors,
                                patch_size = patch_size, 
                                device = device)
            except:
                desc_dict = extract_numpyinput_descriptors(model,
                                desc_name + '_3rdparty' ,
                                subset= subset, 
                                path_to_save_dataset = path_to_save_dataset,
                                path_to_save_descriptors = path_to_save_descriptors,
                                patch_size = patch_size)
            mAP = evaluate_mAP_snn_based(desc_dict['descriptors'],
                         desc_dict['labels'], 
                         desc_dict['img_idxs'],
                         path_to_save_mAP=f'{path_to_save_mAP}/{desc_name}_PS{patch_size}_3rdparty_{subset}.npy',
                         backend=backend,
                         distance=distance)
            results['3rdparty'][subset] = mAP
            print (f'{desc_name} trained on 3rdparty PS = {patch_size} mAP on {subset} = {mAP:.5f}')        
    return results    

In [ ]:
#export
from typing import Dict
def nice_results_3rdparty(desc_name:str, res_dict:Dict):
    '''Returns formatted string with results'''
    if 'liberty' in res_dict:
        lib = f'{(100*res_dict["liberty"]):.2f}'
    else:
        lib = '-----'
    if 'notredame' in res_dict:
        notre = f'{(100*res_dict["notredame"]):.2f}'
    else:
        notre = '-----'
    if 'yosemite' in res_dict:
        yos = f'{(100*res_dict["yosemite"]):.2f}'
    else:
        yos = '-----'
    res = f'{desc_name[:20].ljust(20)}   {yos}              {notre}               {lib} '
    return res


def nice_results_Brown(desc_name:str, res_dict:Dict) -> str:
    '''Returns formatted string with results'''
    NA = '-----'
    lib_yos, lib_notre, yos_notre, yos_lib, notre_lib, notre_yos = NA,NA,NA,NA,NA,NA
    if 'liberty' in res_dict:
        cr = res_dict['liberty']
        if 'notredame' in cr:
            lib_notre = f'{(100*cr["notredame"]):.2f}'
        else:
            lib_notre = NA
        if 'yosemite' in cr:
            lib_yos = f'{(100*cr["yosemite"]):.2f}'
        else:
            lib_yos = NA
    if 'notredame' in res_dict:
        cr = res_dict['notredame']
        if 'liberty' in cr:
            notre_lib = f'{(100*cr["liberty"]):.2f}'
        else:
            notre_lib = NA
        if 'yosemite' in cr:
            notre_yos = f'{(100*cr["yosemite"]):.2f}'
        else:
            notre_yos = NA    
    if 'yosemite' in res_dict:
        cr = res_dict['yosemite']
        if 'liberty' in cr:
            yos_lib = f'{(100*cr["liberty"]):.2f}'
        else:
            yos_lib = NA
        if 'notredame' in cr:
            yos_notre = f'{(100*cr["notredame"]):.2f}'
        else:
            yos_notre = NA    
    
    res = f'{desc_name[:20].ljust(18)} {lib_yos}  {notre_yos}        {lib_notre}  {yos_notre}        {notre_lib}  {yos_lib}'
    return res

def print_results_table(full_res_dict: Dict):
    '''Function, which prints nicely formatted table with all results'''
    TITLE00 = 'Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited'
    sep = '------------------------------------------------------------------------------'
    TITLE1 = 'trained on       liberty notredame  liberty yosemite  notredame yosemite'
    TITLE2 = 'tested  on           yosemite           notredame            liberty'
    print (sep)
    print (TITLE00)
    print (sep)
    print (TITLE1)
    print (TITLE2)
    print (sep)
    for desc_name, desc_results in full_res_dict.items():
        if '3rdparty' in desc_results:
            if len(desc_results['3rdparty']) == 3:
                print (nice_results_3rdparty(desc_name, desc_results['3rdparty']))
            else:
                print (nice_results_Brown(desc_name, desc_results))
        else:
            print (nice_results_Brown(desc_name, desc_results))
    print (sep)
    return

Some visualization

In [ ]:
res = {'Kornia RootSIFT 32px': 
        {'3rdparty': {'liberty': 0.49652328,
                      'notredame': 0.49066364,
                      'yosemite': 0.58237198}},
       'OpenCV_LATCH 65px': 
        {'yosemite': {'liberty': 0.39075459, 
                      'notredame': 0.37258606}}}
print_results_table(res)

Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited
------------------------------------------------------------------------------
trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
------------------------------------------------------------------------------
Kornia RootSIFT 32px   58.24              49.07               49.65 
OpenCV_LATCH 65px  -----  -----        -----  37.26        -----  39.08
------------------------------------------------------------------------------
